In [1]:
import toml
import json
import millify
import gspread
import requests
import calendar
import numpy as np
import pandas as pd
import polars as pl
import datetime as dt
import streamlit as st
from millify import prettify
import chart_functions as chart
from lxml.html import fromstring
from streamlit_gsheets import GSheetsConnection
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
scope = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

def authenticate(secret_path, scope):
    credentials_file = json.loads(str(toml.load(secret_path)['connections']['gsheets']).replace("'", '"').replace('\r\n', '\\r\\n'))
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(credentials_file, scopes=scope)
    client = gspread.authorize(credentials)
    wb = client.open('NLFB')
    return wb

WORKBOOK = authenticate('.streamlit/secrets.toml', scope)


In [37]:
with open('.streamlit/secrets.toml', 'r') as f:
    file = f.read()
    json.loads(file)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [12]:
sheet = worksheet_list[1]

In [18]:
sheet.get_values()[5]

['30',
 '9781529111798',
 'February',
 '2024',
 'I who have never known men',
 '7.51',
 'Jacqueline Harpman',
 'Vintage Publishing',
 '208',
 'Female',
 '1997',
 '',
 '',
 '',
 'no',
 'yes',
 '']

In [26]:
newd = df.group_by('Publisher').agg(pl.col("Score").mean(), pl.col("Title").count())

In [61]:
import calendar
abbr_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}

df = df.with_columns(
    pl.col('Score').cast(pl.Float64),
    pl.col('Pages').cast(pl.Int64),
    pl.col('Goodreads score').cast(pl.Float64),
    pl.col('Year').cast(pl.Int32),
    pl.col('Month').replace_strict(abbr_to_num).alias("Month Num"),
    
)

df = df.with_columns(
    pl.date(pl.col('Year'), pl.col('Month Num'), 1).alias("Date")
)

In [67]:
import streamlit as st
import datetime as dt



st.metric(
    label="Total pages read",
    value=df['Pages'].sum(),
    delta=df.filter(pl.col('Date') > (dt.date.today() - dt.timedelta(days=28)))['Pages'].sum(), 
    delta_color="inverse"
)

2024-08-24 22:09:11.311 
  command:

    streamlit run c:\ProgramData\Anaconda3\envs\NLFB\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [69]:
import numpy as np
np.std(df['Score'].drop_nans().to_numpy())

nan

In [76]:
np.std(df['Score'].drop_nulls().to_numpy())

0.8243643519555455

In [42]:
px.bar(newd, x='Publisher', y='Score', secondary_y)

In [55]:
import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Bar(name='Score', x=newd['Publisher'], y=newd['Score'], yaxis='y1', offsetgroup=1),
    go.Bar(name='Book Count', x=newd['Publisher'], y=newd['Title'], yaxis='y2', offsetgroup=2)
],
layout={
        'yaxis': {'title': 'Score'},
        'yaxis2': {'title': 'Book Count', 'overlaying': 'y', 'side': 'right'}
    }
)
# Change the bar mode
#fig.update_layout(barmode='group')
fig.show()

In [25]:
df['Year'] = df.Date.apply(lambda x: x.year)

In [26]:
df.Year.unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014],
      dtype=int64)

In [1]:
l = [None]
l * 0

[]

In [1]:
import requests
site = requests.get("https://www.meetup.com/20-and-30-somethings-book-club-london/")

In [2]:
type(site)

requests.models.Response

In [27]:
import requests
from lxml.html import fromstring

url = "https://www.meetup.com/20-and-30-somethings-book-club-london/"  # Replace with the desired URL
response = requests.get(url)

soup = fromstring(response.text)
element = soup.get_element_by_id("member-count-link")
text = str(element.text_content())
text.split(" · ")[0]



'5,928 members'

'5,928 members · Public group'